# Base de dados usada para a realização desse projeto

[COVID-19 Vaccinations in the United States, County](https://catalog.data.gov/dataset/covid-19-vaccinations-in-the-united-statescounty-8204e/resource/5c3af69e-46fc-494e-888b-63cc36845ede)

In [ ]:
#Instalando a Biblioteca Pandera (no Colab toda vez em que se inicia o notebook tem que fazer a instalação da bibliotera para ser utilizada)

!pip install pandera

# Importando Bibliotecas para a utlização do pandas e para acessar o bucket do GCP

In [ ]:
from google.cloud import storage
import pandas as pd
import pandera as pa
import io
from io import BytesIO

In [ ]:
# Criando o client object

storage_client = storage.Client.from_service_account_json("/content/chave_projeto-finalsc01-b94741ddd471.json")

# Acessando o Bucket do Projeto

In [ ]:
BUCKET_NAME = 'projeto-finalsc01'

bucket = storage_client.get_bucket(BUCKET_NAME)

# Listando os arquivos contidos no Bucket

In [ ]:
filename  = list(bucket.list_blobs(prefix=''))
for name in filename:
  print(name.name)

chave/
chave/projeto-finalsc01-b94741ddd471.json
dados-brutos/
dados-brutos/COVID-19_Vaccinations_in_the_United_States_County.csv
dados-brutos/COVID_US_HOSPITALIZATION.json
dados-tratados/
dados-tratados/Casos_Hospitalizacoes_Obitos_Cidade.csv


# Carregando o dataset .csv

In [ ]:
blop = bucket.blob("dados-brutos/COVID-19_Vaccinations_in_the_United_States_County.csv")
data = blop.download_as_string()

# Criando um DataFrame utilizando pandas

In [ ]:
df = pd.read_csv(io.BytesIO(data), encoding='utf-8', sep=',', parse_dates = ["Date"], dayfirst = True)

df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Date,FIPS,MMWR_week,Recip_County,Recip_State,Series_Complete_Pop_Pct,Series_Complete_Yes,Series_Complete_12Plus,Series_Complete_12PlusPop_Pct,Series_Complete_18Plus,Series_Complete_18PlusPop_Pct,Series_Complete_65Plus,Series_Complete_65PlusPop_Pct,Completeness_pct,Administered_Dose1_Recip,Administered_Dose1_Pop_Pct,Administered_Dose1_Recip_12Plus,Administered_Dose1_Recip_12PlusPop_Pct,Administered_Dose1_Recip_18Plus,Administered_Dose1_Recip_18PlusPop_Pct,Administered_Dose1_Recip_65Plus,Administered_Dose1_Recip_65PlusPop_Pct,SVI_CTGY,Series_Complete_Pop_Pct_SVI,Series_Complete_12PlusPop_Pct_SVI,Series_Complete_18PlusPop_Pct_SVI,Series_Complete_65PlusPop_Pct_SVI,Metro_status,Series_Complete_Pop_Pct_UR_Equity,Series_Complete_12PlusPop_Pct_UR_Equity,Series_Complete_18PlusPop_Pct_UR_Equity,Series_Complete_65PlusPop_Pct_UR_Equity
0,2021-11-17,19149,46,Plymouth County,IA,48.3,12168,12168.0,57.6,11528,61.1,3917,83.9,97.4,12921.0,51.3,12857.0,60.9,12175.0,64.5,4077.0,87.4,A,3.0,4.0,4.0,4.0,Metro,3.0,4.0,4.0,4.0
1,2021-11-17,01021,46,Chilton County,AL,31.9,14178,14177.0,37.8,13447,39.7,4688,61.7,92.6,17431.0,39.2,17413.0,46.5,16523.0,48.8,5480.0,72.1,D,14.0,14.0,14.0,15.0,Metro,2.0,2.0,2.0,3.0
2,2021-11-17,20167,46,Russell County,KS,39.8,2727,2727.0,46.8,2636,49.1,1112,65.6,93.6,3013.0,43.9,3002.0,51.6,2894.0,53.9,1197.0,70.7,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0
3,2021-11-17,36031,46,Essex County,NY,65.8,24262,24262.0,73.1,22876,73.8,7418,82.9,97.6,25992.0,70.5,25723.0,77.5,24189.0,78.1,7853.0,87.8,B,8.0,8.0,8.0,8.0,Non-metro,8.0,8.0,8.0,8.0
4,2021-11-17,16017,46,Bonner County,ID,37.8,17308,NaN,NaN,17267,47.0,7946,68.4,97.6,19484.0,42.6,NaN,NaN,19428.0,52.9,8714.0,75.0,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119189,2020-12-13,22123,51,West Carroll Parish,LA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
1119190,2020-12-13,13293,51,Upson County,GA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,D,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
1119191,2020-12-13,48137,51,Edwards County,TX,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,B,NaN,NaN,NaN,NaN,Non-metro,NaN,NaN,NaN,NaN
1119192,2020-12-13,13143,51,Haralson County,GA,0.0,0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,C,NaN,NaN,NaN,NaN,Metro,NaN,NaN,NaN,NaN


# Analisando a base de dados

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1119194 entries, 0 to 1119193
Data columns (total 32 columns):
 #   Column                                   Non-Null Count    Dtype         
---  ------                                   --------------    -----         
 0   Date                                     1119194 non-null  datetime64[ns]
 1   FIPS                                     1119194 non-null  object        
 2   MMWR_week                                1119194 non-null  int64         
 3   Recip_County                             1119194 non-null  object        
 4   Recip_State                              1119194 non-null  object        
 5   Series_Complete_Pop_Pct                  1119194 non-null  float64       
 6   Series_Complete_Yes                      1119194 non-null  int64         
 7   Series_Complete_12Plus                   1103848 non-null  float64       
 8   Series_Complete_12PlusPop_Pct            1103848 non-null  float64       
 9   Series_Comple

Podemos verificar que a base de dados contém 1 coluna do tipo datetime, 22 colunas do tipo float, 4 colunas do tipo int e 5 colunas do tipo string

Podemos verificar também que algumas colunas contém dados faltantes

# Traduzindo o Schema para Português-BR

In [ ]:
df.rename(columns={'Date': 'Data',
                   'FIPS': 'CEPPIF', #Código Estadual Padrão de Processamento de Informações Federais
                   'MMWR_week': 'Relatorio_semanal_morbidade_mortalidade', #controle e prevenção de doenças
                   'Recip_County': 'Condado', #região em que reside o paciente
                   'Recip_State': 'Estado', #estado em que reside o paciente
                   'Series_Complete_Pop_Pct': 'Porcentagem_populacao_vacinada', #Porcentagem de pessoas que estão totalmente vacinadas (têm segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição e no condado onde vive o paciente
                   'Series_Complete_Yes': 'Numero_total_vacinados', #Número total de pessoas que estão totalmente vacinadas (têm segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição e no condado onde vive o paciente
                   'Series_Complete_12Plus': 'Numero_total_12+', #Número total de pessoas com mais de 12 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição onde vive o paciente
                   'Series_Complete_12PlusPop_Pct': 'Porcentagem_total_12+', #Porcentagem de pessoas com mais de 12 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição onde vive o paciente
                   'Series_Complete_18Plus': 'Numero_total_18+', #Número total de pessoas com mais de 18 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição e no condado onde vive o paciente
                   'Series_Complete_18PlusPop_Pct': 'Porcentagem_total_18+', #Porcentagem de pessoas com mais de 18 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição e no condado onde vive o paciente
                   'Series_Complete_65Plus': 'Numero_total_65+', #Número total de pessoas com mais de 65 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição onde vive o paciente
                   'Series_Complete_65PlusPop_Pct': 'Porcentagem_total_65+', #Porcentagem de pessoas com mais de 65 anos que estão totalmente vacinadas (têm uma segunda dose de uma vacina de duas doses ou uma dose de uma vacina de dose única) com base na jurisdição onde vive o paciente
                   'Completeness_pct': 'Porcentagem_total_pessoas_vacinadas', #Representa a proporção de pessoas totalmente vacinadas cujo código Federal Information Processing Standards (FIPS) é relatado e corresponde a um código FIPS de condado válido na jurisdição.
                   'Administered_Dose1_Recip': 'Primeira_dose', #Pessoas com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Pop_Pct': 'Porcentagem_primeira_dose', #Porcentagem da População total com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_12Plus': 'Primeira_dose_12+', #Pessoas com mais de 12 anos com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_12PlusPop_Pct': 'Porcentagem_primeira_dose_12+', #Porcentagem de 12+ Pop com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_18Plus': 'Primeira_dose_18+', #Pessoas com mais de 18 anos com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_18PlusPop_Pct': 'Porcentagem_primeira_dose_18+', #Porcentagem de Pop 18+ com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_65Plus': 'Primeira_dose_65+', #Pessoas com mais de 65 anos com pelo menos uma dose por estado de residência
                   'Administered_Dose1_Recip_65PlusPop_Pct': 'Porcentagem_primeira_dose_65+', #Porcentagem de Pop com mais de 65 anos com pelo menos uma dose por estado de residência
                   'SVI_CTGY': 'IVS_Categoria', #Índice de Vulnerabilidade Social por Categoria
                   'Series_Complete_Pop_Pct_SVI': 'Porcentagem_total_pop_vacinada', #Porcentagem da população totalmente vacinada / IVS
                   'Series_Complete_12PlusPop_Pct_SVI': 'Porcentagem_total_12+_IVS', #Porcentagem da população com 12+ totalmente vacinadas / IVS
                   'Series_Complete_18PlusPop_Pct_SVI': 'Porcentagem_total_18+_IVS', #Porcentagem da população com 18+ totalmente vacinadas / IVS
                   'Series_Complete_65PlusPop_Pct_SVI': 'Porcentagem_total_65+_IVS', #Porcentagem da população com 65+ totalmente vacinadas / IVS
                   'Metro_status': 'Status_metropolitano', #Status da área metropolitana (se é uma área metropolitana ou não metropolitana)
                   'Series_Complete_Pop_Pct_UR_Equity': 'Porcentagem_total_metropolitana/NaoMetropolitana', #Porcentagem da população totalmente vacinada - Metropolitana / Não Metropolitana
                   'Series_Complete_12PlusPop_Pct_UR_Equity': 'Porcentagem_total_12+_metropolitana/NaoMetropolitana',  #Porcentagem da população com 12+ totalmente vacinada - Metropolitana / Não Metropolitana
                   'Series_Complete_18PlusPop_Pct_UR_Equity': 'Porcentagem_total_18+_metropolitana/NaoMetropolitana', #Porcentagem da população com 18+ totalmente vacinada - Metropolitana / Não Metropolitana
                   'Series_Complete_65PlusPop_Pct_UR_Equity': 'Porcentagem_total_65+_metropolitana/NaoMetropolitana' #Porcentagem da população com 65+ totalmente vacinada - Metropolitana / Não Metropolitana

                   }, inplace = True)

In [ ]:
df.head()

,Data,CEPPIF,Relatorio_semanal_morbidade_mortalidade,Condado,Estado,Porcentagem_populacao_vacinada,Numero_total_vacinados,Numero_total_12+,Porcentagem_total_12+,Numero_total_18+,Porcentagem_total_18+,Numero_total_65+,Porcentagem_total_65+,Porcentagem_total_pessoas_vacinadas,Primeira_dose,Porcentagem_primeira_dose,Primeira_dose_12+,Porcentagem_primeira_dose_12+,Primeira_dose_18+,Porcentagem_primeira_dose_18+,Primeira_dose_65+,Porcentagem_primeira_dose_65+,IVS_Categoria,Porcentagem_total_pop_vacinada,Porcentagem_total_12+_IVS,Porcentagem_total_18+_IVS,Porcentagem_total_65+_IVS,Status_metropolitano,Porcentagem_total_metropolitana/NaoMetropolitana,Porcentagem_total_12+_metropolitana/NaoMetropolitana,Porcentagem_total_18+_metropolitana/NaoMetropolitana,Porcentagem_total_65+_metropolitana/NaoMetropolitana
0,2021-11-17,19149,46,Plymouth County,IA,48.3,12168,12168.0,57.6,11528,61.1,3917,83.9,97.4,12921.0,51.3,12857.0,60.9,12175.0,64.5,4077.0,87.4,A,3.0,4.0,4.0,4.0,Metro,3.0,4.0,4.0,4.0
1,2021-11-17,01021,46,Chilton County,AL,31.9,14178,14177.0,37.8,13447,39.7,4688,61.7,92.6,17431.0,39.2,17413.0,46.5,16523.0,48.8,5480.0,72.1,D,14.0,14.0,14.0,15.0,Metro,2.0,2.0,2.0,3.0
2,2021-11-17,20167,46,Russell County,KS,39.8,2727,2727.0,46.8,2636,49.1,1112,65.6,93.6,3013.0,43.9,3002.0,51.6,2894.0,53.9,1197.0,70.7,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0
3,2021-11-17,36031,46,Essex County,NY,65.8,24262,24262.0,73.1,22876,73.8,7418,82.9,97.6,25992.0,70.5,25723.0,77.5,24189.0,78.1,7853.0,87.8,B,8.0,8.0,8.0,8.0,Non-metro,8.0,8.0,8.0,8.0
4,2021-11-17,16017,46,Bonner County,ID,37.8,17308,NaN,NaN,17267,47.0,7946,68.4,97.6,19484.0,42.6,NaN,NaN,19428.0,52.9,8714.0,75.0,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0


# Verificando se a coluna **Relatorio_semanal_morbidade_mortalidade** contém valores únicos

In [ ]:
#Verificação realizada para saber o que cada código significava
pd.unique(df['Relatorio_semanal_morbidade_mortalidade'])

array([46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30,
       29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13,
       12, 11, 10,  9,  8,  7,  6,  5,  4,  3,  2,  1, 53, 52, 51])

# Deletando colunas que não fazem sentido para a análise de dados

In [ ]:
df.drop(['CEPPIF', 'Relatorio_semanal_morbidade_mortalidade', 'Porcentagem_populacao_vacinada'], axis = 1, inplace = True)



1.   A coluna **CEPPIF** representa o código da região federal, como já temos o condado e o estado, não será necessária na base de dados.

2.   Como não foi encontrado o que representa cada código da coluna **Relatorio_semanal_morbidade_mortalidade**, resolvemos dropar para não atrapalhar a análise de dados.

3.   A coluna **Porcentagem_populacao_vacinada** faz referência a coluna **Relatorio_semanal_morbidade_mortalidade**, então não faz sentido deixar se não sabemos o que representa cada código da coluna de relatório.

In [ ]:
df.head()

,Data,Condado,Estado,Numero_total_vacinados,Numero_total_12+,Porcentagem_total_12+,Numero_total_18+,Porcentagem_total_18+,Numero_total_65+,Porcentagem_total_65+,Porcentagem_total_pessoas_vacinadas,Primeira_dose,Porcentagem_primeira_dose,Primeira_dose_12+,Porcentagem_primeira_dose_12+,Primeira_dose_18+,Porcentagem_primeira_dose_18+,Primeira_dose_65+,Porcentagem_primeira_dose_65+,IVS_Categoria,Porcentagem_total_pop_vacinada,Porcentagem_total_12+_IVS,Porcentagem_total_18+_IVS,Porcentagem_total_65+_IVS,Status_metropolitano,Porcentagem_total_metropolitana/NaoMetropolitana,Porcentagem_total_12+_metropolitana/NaoMetropolitana,Porcentagem_total_18+_metropolitana/NaoMetropolitana,Porcentagem_total_65+_metropolitana/NaoMetropolitana
0,2021-11-17,Plymouth County,IA,12168,12168.0,57.6,11528,61.1,3917,83.9,97.4,12921.0,51.3,12857.0,60.9,12175.0,64.5,4077.0,87.4,A,3.0,4.0,4.0,4.0,Metro,3.0,4.0,4.0,4.0
1,2021-11-17,Chilton County,AL,14178,14177.0,37.8,13447,39.7,4688,61.7,92.6,17431.0,39.2,17413.0,46.5,16523.0,48.8,5480.0,72.1,D,14.0,14.0,14.0,15.0,Metro,2.0,2.0,2.0,3.0
2,2021-11-17,Russell County,KS,2727,2727.0,46.8,2636,49.1,1112,65.6,93.6,3013.0,43.9,3002.0,51.6,2894.0,53.9,1197.0,70.7,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0
3,2021-11-17,Essex County,NY,24262,24262.0,73.1,22876,73.8,7418,82.9,97.6,25992.0,70.5,25723.0,77.5,24189.0,78.1,7853.0,87.8,B,8.0,8.0,8.0,8.0,Non-metro,8.0,8.0,8.0,8.0
4,2021-11-17,Bonner County,ID,17308,NaN,NaN,17267,47.0,7946,68.4,97.6,19484.0,42.6,NaN,NaN,19428.0,52.9,8714.0,75.0,A,2.0,3.0,3.0,3.0,Non-metro,6.0,7.0,7.0,7.0


# Verificando os valores nulos

In [ ]:
df.isna().sum()

Data                                                         0
Condado                                                      0
Estado                                                       0
Numero_total_vacinados                                       0
Numero_total_12+                                         15346
Porcentagem_total_12+                                    15346
Numero_total_18+                                             0
Porcentagem_total_18+                                        0
Numero_total_65+                                             0
Porcentagem_total_65+                                        0
Porcentagem_total_pessoas_vacinadas                          0
Primeira_dose                                            35189
Porcentagem_primeira_dose                                    0
Primeira_dose_12+                                        76770
Porcentagem_primeira_dose_12+                            15346
Primeira_dose_18+                                      

# Deletando linhas com dados faltantes

In [ ]:
#Verificando os valores conditos na coluna 'Status_metropolitano'
pd.unique(df['Status_metropolitano'])

array(['Metro', 'Non-metro', nan], dtype=object)

In [ ]:
#Deletando valores nulos da coluna 'Status_metropolitano'
df.dropna(subset = ['Status_metropolitano'], inplace = True)

In [ ]:
#Traduzindo os valores conditos na coluna 'Status_metropolitano'

df.loc[df.Status_metropolitano == 'Metro', ['Status_metropolitano']] = 'Metropolitano'
df.loc[df.Status_metropolitano == 'Non-metro', ['Status_metropolitano']] = 'Não Metropolitano'

df.head()

,Data,Condado,Estado,Numero_total_vacinados,Numero_total_12+,Porcentagem_total_12+,Numero_total_18+,Porcentagem_total_18+,Numero_total_65+,Porcentagem_total_65+,Porcentagem_total_pessoas_vacinadas,Primeira_dose,Porcentagem_primeira_dose,Primeira_dose_12+,Porcentagem_primeira_dose_12+,Primeira_dose_18+,Porcentagem_primeira_dose_18+,Primeira_dose_65+,Porcentagem_primeira_dose_65+,IVS_Categoria,Porcentagem_total_pop_vacinada,Porcentagem_total_12+_IVS,Porcentagem_total_18+_IVS,Porcentagem_total_65+_IVS,Status_metropolitano,Porcentagem_total_metropolitana/NaoMetropolitana,Porcentagem_total_12+_metropolitana/NaoMetropolitana,Porcentagem_total_18+_metropolitana/NaoMetropolitana,Porcentagem_total_65+_metropolitana/NaoMetropolitana
0,2021-11-17,Plymouth County,IA,12168,12168.0,57.6,11528,61.1,3917,83.9,97.4,12921.0,51.3,12857.0,60.9,12175.0,64.5,4077.0,87.4,A,3.0,4.0,4.0,4.0,Metropolitano,3.0,4.0,4.0,4.0
1,2021-11-17,Chilton County,AL,14178,14177.0,37.8,13447,39.7,4688,61.7,92.6,17431.0,39.2,17413.0,46.5,16523.0,48.8,5480.0,72.1,D,14.0,14.0,14.0,15.0,Metropolitano,2.0,2.0,2.0,3.0
2,2021-11-17,Russell County,KS,2727,2727.0,46.8,2636,49.1,1112,65.6,93.6,3013.0,43.9,3002.0,51.6,2894.0,53.9,1197.0,70.7,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0
3,2021-11-17,Essex County,NY,24262,24262.0,73.1,22876,73.8,7418,82.9,97.6,25992.0,70.5,25723.0,77.5,24189.0,78.1,7853.0,87.8,B,8.0,8.0,8.0,8.0,Não Metropolitano,8.0,8.0,8.0,8.0
4,2021-11-17,Bonner County,ID,17308,NaN,NaN,17267,47.0,7946,68.4,97.6,19484.0,42.6,NaN,NaN,19428.0,52.9,8714.0,75.0,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0


In [ ]:
#Verificando os valores conditos na coluna 'IVS_Categoria'
pd.unique(df['IVS_Categoria'])

array(['A', 'D', 'B', 'C', nan], dtype=object)

In [ ]:
#Deletando valores nulos da coluna 'IVS_Categoria'
df.dropna(subset = ['IVS_Categoria'], inplace = True)

In [ ]:
#Conferindo se os valores nulos das colunas 'Status_metropolitano' e 'IVS_Categoria' foram deletados
df.isna().sum()

Data                                                         0
Condado                                                      0
Estado                                                       0
Numero_total_vacinados                                       0
Numero_total_12+                                         15007
Porcentagem_total_12+                                    15007
Numero_total_18+                                             0
Porcentagem_total_18+                                        0
Numero_total_65+                                             0
Porcentagem_total_65+                                        0
Porcentagem_total_pessoas_vacinadas                          0
Primeira_dose                                            34260
Porcentagem_primeira_dose                                    0
Primeira_dose_12+                                        73867
Porcentagem_primeira_dose_12+                            15007
Primeira_dose_18+                                      

# Substituindo valores nulos por 0

In [ ]:
#Substituindo os valores nulos das colunas restantes por 0 para facilitar a análise estatística do DataFrame
df.fillna(0, inplace = True)

In [ ]:
df.head()

,Data,Condado,Estado,Numero_total_vacinados,Numero_total_12+,Porcentagem_total_12+,Numero_total_18+,Porcentagem_total_18+,Numero_total_65+,Porcentagem_total_65+,Porcentagem_total_pessoas_vacinadas,Primeira_dose,Porcentagem_primeira_dose,Primeira_dose_12+,Porcentagem_primeira_dose_12+,Primeira_dose_18+,Porcentagem_primeira_dose_18+,Primeira_dose_65+,Porcentagem_primeira_dose_65+,IVS_Categoria,Porcentagem_total_pop_vacinada,Porcentagem_total_12+_IVS,Porcentagem_total_18+_IVS,Porcentagem_total_65+_IVS,Status_metropolitano,Porcentagem_total_metropolitana/NaoMetropolitana,Porcentagem_total_12+_metropolitana/NaoMetropolitana,Porcentagem_total_18+_metropolitana/NaoMetropolitana,Porcentagem_total_65+_metropolitana/NaoMetropolitana
0,2021-11-17,Plymouth County,IA,12168,12168.0,57.6,11528,61.1,3917,83.9,97.4,12921.0,51.3,12857.0,60.9,12175.0,64.5,4077.0,87.4,A,3.0,4.0,4.0,4.0,Metropolitano,3.0,4.0,4.0,4.0
1,2021-11-17,Chilton County,AL,14178,14177.0,37.8,13447,39.7,4688,61.7,92.6,17431.0,39.2,17413.0,46.5,16523.0,48.8,5480.0,72.1,D,14.0,14.0,14.0,15.0,Metropolitano,2.0,2.0,2.0,3.0
2,2021-11-17,Russell County,KS,2727,2727.0,46.8,2636,49.1,1112,65.6,93.6,3013.0,43.9,3002.0,51.6,2894.0,53.9,1197.0,70.7,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0
3,2021-11-17,Essex County,NY,24262,24262.0,73.1,22876,73.8,7418,82.9,97.6,25992.0,70.5,25723.0,77.5,24189.0,78.1,7853.0,87.8,B,8.0,8.0,8.0,8.0,Não Metropolitano,8.0,8.0,8.0,8.0
4,2021-11-17,Bonner County,ID,17308,0.0,0.0,17267,47.0,7946,68.4,97.6,19484.0,42.6,0.0,0.0,19428.0,52.9,8714.0,75.0,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0


In [ ]:
#Verificando se os valores nulos foram substituídos
df.isna().sum()

Data                                                    0
Condado                                                 0
Estado                                                  0
Numero_total_vacinados                                  0
Numero_total_12+                                        0
Porcentagem_total_12+                                   0
Numero_total_18+                                        0
Porcentagem_total_18+                                   0
Numero_total_65+                                        0
Porcentagem_total_65+                                   0
Porcentagem_total_pessoas_vacinadas                     0
Primeira_dose                                           0
Porcentagem_primeira_dose                               0
Primeira_dose_12+                                       0
Porcentagem_primeira_dose_12+                           0
Primeira_dose_18+                                       0
Porcentagem_primeira_dose_18+                           0
Primeira_dose_

# Modificando o tipo de dado de algumas colunas para fazer a validação

In [ ]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1097549 entries, 0 to 1119193
Data columns (total 29 columns):
 #   Column                                                Non-Null Count    Dtype         
---  ------                                                --------------    -----         
 0   Data                                                  1097549 non-null  datetime64[ns]
 1   Condado                                               1097549 non-null  object        
 2   Estado                                                1097549 non-null  object        
 3   Numero_total_vacinados                                1097549 non-null  int64         
 4   Numero_total_12+                                      1097549 non-null  float64       
 5   Porcentagem_total_12+                                 1097549 non-null  float64       
 6   Numero_total_18+                                      1097549 non-null  int64         
 7   Porcentagem_total_18+                                 

In [ ]:
#Modificando a coluna 'Numero_total_12+' de float para int
df[['Numero_total_12+']] = df[['Numero_total_12+']].astype(int)

#Modificando a coluna 'Primeira_dose' de float para int
df[['Primeira_dose']] = df[['Primeira_dose']].astype(int)

#Modificando a coluna 'Primeira_dose_12+' de float para int
df[['Primeira_dose_12+']] = df[['Primeira_dose_12+']].astype(int)

#Modificando a coluna 'Primeira_dose_18+' de float para int
df[['Primeira_dose_18+']] = df[['Primeira_dose_18+']].astype(int)

#Modificando a coluna 'Primeira_dose_65+' de float para int
df[['Primeira_dose_65+']] = df[['Primeira_dose_65+']].astype(int)

# Validando os dados do DataFrame usando a biblioteca panderas

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "Data":pa.Column(pa.DateTime),
        "Condado":pa.Column(pa.String),
        "Estado":pa.Column(pa.String),
        "Numero_total_vacinados":pa.Column(pa.Int),
        "Numero_total_12+":pa.Column(pa.Int),
        "Porcentagem_total_12+":pa.Column(pa.Float),
        "Numero_total_18+":pa.Column(pa.Int),
        "Porcentagem_total_18+":pa.Column(pa.Float),
        "Numero_total_65+":pa.Column(pa.Int),
        "Porcentagem_total_65+":pa.Column(pa.Float),
        "Porcentagem_total_pessoas_vacinadas":pa.Column(pa.Float),
        "Primeira_dose":pa.Column(pa.Int),
        "Porcentagem_primeira_dose":pa.Column(pa.Float),
        "Primeira_dose_12+":pa.Column(pa.Int),
        "Porcentagem_primeira_dose_12+":pa.Column(pa.Float),
        "Primeira_dose_18+":pa.Column(pa.Int),
        "Porcentagem_primeira_dose_18+":pa.Column(pa.Float),
        "Primeira_dose_65+":pa.Column(pa.Int),
        "Porcentagem_primeira_dose_65+":pa.Column(pa.Float),
        "IVS_Categoria":pa.Column(pa.String),
        "Porcentagem_total_pop_vacinada":pa.Column(pa.Float),
        "Porcentagem_total_12+_IVS":pa.Column(pa.Float),
        "Porcentagem_total_18+_IVS":pa.Column(pa.Float),
        "Porcentagem_total_65+_IVS":pa.Column(pa.Float),
        "Status_metropolitano":pa.Column(pa.String),
        "Porcentagem_total_metropolitana/NaoMetropolitana":pa.Column(pa.Float),
        "Porcentagem_total_12+_metropolitana/NaoMetropolitana":pa.Column(pa.Float),
        "Porcentagem_total_18+_metropolitana/NaoMetropolitana":pa.Column(pa.Float),
        "Porcentagem_total_65+_metropolitana/NaoMetropolitana":pa.Column(pa.Float)

    }
)

schema.validate(df)

,Data,Condado,Estado,Numero_total_vacinados,Numero_total_12+,Porcentagem_total_12+,Numero_total_18+,Porcentagem_total_18+,Numero_total_65+,Porcentagem_total_65+,Porcentagem_total_pessoas_vacinadas,Primeira_dose,Porcentagem_primeira_dose,Primeira_dose_12+,Porcentagem_primeira_dose_12+,Primeira_dose_18+,Porcentagem_primeira_dose_18+,Primeira_dose_65+,Porcentagem_primeira_dose_65+,IVS_Categoria,Porcentagem_total_pop_vacinada,Porcentagem_total_12+_IVS,Porcentagem_total_18+_IVS,Porcentagem_total_65+_IVS,Status_metropolitano,Porcentagem_total_metropolitana/NaoMetropolitana,Porcentagem_total_12+_metropolitana/NaoMetropolitana,Porcentagem_total_18+_metropolitana/NaoMetropolitana,Porcentagem_total_65+_metropolitana/NaoMetropolitana
0,2021-11-17,Plymouth County,IA,12168,12168,57.6,11528,61.1,3917,83.9,97.4,12921,51.3,12857,60.9,12175,64.5,4077,87.4,A,3.0,4.0,4.0,4.0,Metropolitano,3.0,4.0,4.0,4.0
1,2021-11-17,Chilton County,AL,14178,14177,37.8,13447,39.7,4688,61.7,92.6,17431,39.2,17413,46.5,16523,48.8,5480,72.1,D,14.0,14.0,14.0,15.0,Metropolitano,2.0,2.0,2.0,3.0
2,2021-11-17,Russell County,KS,2727,2727,46.8,2636,49.1,1112,65.6,93.6,3013,43.9,3002,51.6,2894,53.9,1197,70.7,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0
3,2021-11-17,Essex County,NY,24262,24262,73.1,22876,73.8,7418,82.9,97.6,25992,70.5,25723,77.5,24189,78.1,7853,87.8,B,8.0,8.0,8.0,8.0,Não Metropolitano,8.0,8.0,8.0,8.0
4,2021-11-17,Bonner County,ID,17308,0,0.0,17267,47.0,7946,68.4,97.6,19484,42.6,0,0.0,19428,52.9,8714,75.0,A,2.0,3.0,3.0,3.0,Não Metropolitano,6.0,7.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119189,2020-12-13,West Carroll Parish,LA,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,C,0.0,0.0,0.0,0.0,Não Metropolitano,0.0,0.0,0.0,0.0
1119190,2020-12-13,Upson County,GA,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,D,0.0,0.0,0.0,0.0,Não Metropolitano,0.0,0.0,0.0,0.0
1119191,2020-12-13,Edwards County,TX,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,B,0.0,0.0,0.0,0.0,Não Metropolitano,0.0,0.0,0.0,0.0
1119192,2020-12-13,Haralson County,GA,0,0,0.0,0,0.0,0,0.0,0.0,0,0.0,0,0.0,0,0.0,0,0.0,C,0.0,0.0,0.0,0.0,Metropolitano,0.0,0.0,0.0,0.0


# Salvando as transformações realizadas no DataFrame para o Bucket do GCP

In [ ]:
bucket = storage_client.get_bucket('projeto-finalsc01')
bucket.blob('dados-tratados/Vacinacoes_nos_EUA.csv').upload_from_string(df.to_csv(index=False),'Vacinacoes_nos_EUA.csv' )